# Pipelines

Pipelines sequentially apply **a list of transforms** and a **final estimator**. Intermediate steps of the pipeline must be ‘transforms’, that is, they must implement fit and transform methods. The final estimator only needs to implement fit.


### Summary

* Pipeline of transforms with a final estimator.

* Sequentially apply a list of transforms and a final estimator. 

    * Intermediate steps of the pipeline must be ‘transforms’, that is, they must implement fit and transform methods. 
    * The final estimator only needs to implement fit. The transformers in the pipeline can be cached using memory argument.

* The purpose of the pipeline is to assemble several steps that can be cross-validated together while setting different parameters. 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

X = iris.data
y = iris.target

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

## Simple Pipeline

The simple pipeline is composed of the folloing steps:
- Transformation
    - Scaling values between 0 and 1
    - PCA (we keep 2 components)
- Estimator
    - Logistic Regression

### Transformation


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

preprocessing_transformer = Pipeline(steps=[('scale_01', MinMaxScaler(feature_range=(0, 1))), 
                                            ('PCA', PCA(n_components=2))])

### Estimator

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', multi_class='auto')

### Creating and evaluating the Pipeline

In [ ]:
from sklearn.metrics import accuracy_score

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessing_transformer', preprocessing_transformer),
                              ('model', model)
                             ], verbose = True)

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = accuracy_score(y_valid, preds)
print('Accuracy Score:', score)

### Analyzing the transformation

In [ ]:
transformed_Dataset = preprocessing_transformer.fit_transform(X_train)

In [ ]:
type(transformed_Dataset)

In [ ]:
tra_df = pd.DataFrame(transformed_Dataset)

In [ ]:
tra_df.shape

In [ ]:
tra_df.head()

In [ ]:
# Simple check

scaled = MinMaxScaler(feature_range=(0, 1))
scaled_X_train = scaled.fit_transform(X_train)
pcaed = PCA(n_components=2)
pca_X_train = pcaed.fit_transform(scaled_X_train)

In [ ]:
pca_X_train

## ColumnTransformer: Managing different kinds of transformers on different columns:
Extracted and extended from a kaggle.com tutorial

Applies transformers to columns of an array or pandas DataFrame.

This estimator allows different columns or column subsets of the input to be transformed separately and the features generated by each transformer will be concatenated to form a single feature space. This is useful for heterogeneous or columnar data, to combine several feature extraction mechanisms or transformations into a single transformer.

In [ ]:
dataset = pd.read_csv("melb_data.csv")
dataset.head(5)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(dataset.iloc[:,1:-1], dataset.iloc[:,-1], train_size=0.8, test_size=0.2,
                                                      random_state=0)

We construct the full pipeline in three steps.
* Step 1: Define Preprocessing Steps
* Step 2: Define the Model
* Step 3: Create and Evaluate the Pipeline

### Step 1: Define Preprocessing Steps

Similar to how a pipeline bundles together preprocessing and modeling steps, we use the ColumnTransformer class to bundle together different preprocessing steps. The code below:
- imputes missing values in numerical data, and
- imputes missing values and applies a one-hot encoding to categorical data.

In [ ]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train.columns if
                    X_train[cname].nunique() < 10 and 
                    X_train[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train.columns if 
                X_train[cname].dtype in ['int64', 'float64']]

In [ ]:

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse = False))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

### Step 2: Define the Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

### Step 3: Create and Evaluate the Pipeline

In [ ]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

### Parameter tuning

Setting parameters of the various steps is enabled by using their names and the parameter name separated by a ‘__’

In [ ]:
# Example using a Grid Search
from sklearn.model_selection import GridSearchCV

parameters = {
    'model__n_estimators': [10,50,100],
    'preprocessor__num__strategy': ['most_frequent','constant'],
    'preprocessor__cat__imputer__strategy': ['most_frequent','constant'],
}

gs_clf = GridSearchCV(my_pipeline, parameters, scoring = 'neg_mean_absolute_error', cv=5, iid=False, n_jobs=-1)

gs_clf.fit(X_train, y_train)

In [ ]:
gs_clf.best_params_

In [ ]:
gs_clf.best_score_

## FeatureUnion: Applying multiple transformers in parallel

Concatenates results of multiple transformer objects.

This estimator applies a list of transformer objects in parallel to the input data, then concatenates the results. This is useful to combine several feature extraction mechanisms into a single transformer.

In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_selection import SelectKBest

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

X = iris.data
y = iris.target

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [ ]:
# This dataset is way too high-dimensional. Better do PCA:
pca = PCA(n_components=2)

# Maybe some original features where good, too?
selection = SelectKBest(k=2)

#Normalizing is always a good choice
scaler = MinMaxScaler(feature_range=(0, 1))

# Build estimator from PCA and Univariate selection:

combined_features = FeatureUnion([("pca", pca), ("univ_select", selection), ("normal", scaler)])

In [ ]:
# Use combined features to transform dataset:
X_features = combined_features.fit(X, y).transform(X)
print("Combined space has", X_features.shape[1], "features")

In [ ]:
model = LogisticRegression()


# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('combined_features', combined_features),
                              ('model', model)
                             ], verbose = True)

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = accuracy_score(y_valid, preds)
print('Accuracy Score:', score)

In [ ]:
# This dataset is way too high-dimensional. Better do PCA:
pca = PCA(n_components=2)

# Maybe some original features where good, too?
selection = SelectKBest(k=2)

#Normalizing is always a good choice
scaler = MinMaxScaler(feature_range=(0, 1))

# Build estimator from PCA and Univariate selection:

combined_features = FeatureUnion([("pca", pca), ("univ_select", selection), ("normal", scaler)], transformer_weights={"pca":0.4, "univ_select":0.2, "normal":0.3})

In [ ]:
model = LogisticRegression()


# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('combined_features', combined_features),
                              ('model', model)
                             ], verbose = True)

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = accuracy_score(y_valid, preds)
print('Accuracy Score:', score)

## FunctionTransformer: Constructs a transformer from an arbitrary callable.

Concatenates results of multiple transformer objects.

This estimator applies a list of transformer objects in parallel to the input data, then concatenates the results. This is useful to combine several feature extraction mechanisms into a single transformer.

In [ ]:
from sklearn.preprocessing import FunctionTransformer

dataset = pd.read_csv("melb_data.csv")
X_train, X_valid, y_train, y_valid = train_test_split(dataset.iloc[:,1:-1], dataset.iloc[:,-1], train_size=0.8, test_size=0.2,
                                                      random_state=0)

#Selecting the numerical columns
def columns_num(X):
    numerical_cols = [cname for cname in X.columns if 
                X[cname].dtype in ['int64', 'float64']]
    return X.loc[:,numerical_cols]

fill_na_transformer = Pipeline(steps=[ ('drop_cols', FunctionTransformer(columns_num, validate=False)),
                                       ('fill_na', SimpleImputer(strategy='most_frequent'))  ])


model = RandomForestRegressor(n_estimators=100, random_state=0)



# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', fill_na_transformer),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)